# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

#### NOTES
This program is not working with SPA web sites.


In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [56]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        # Get links on page
        links = [link.get('href') for link in soup.find_all('a')]
        # self.links = [link for link in links if link]
        # Bug fix to avoid duplicate link to be returned
        self.links = list(dict.fromkeys(link for link in links if link))

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [57]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'mailto:hello@mygroovydomain.com',
 'https://twitter.com/edwarddonner',
 'https://www.facebook.com/edward.donner.52']

In [60]:
print(f"Title: {ed.title}")
print(f"\nText: {ed.text[:100]}")
print(f"\nContent: {ed.get_contents()[:100]}")

Title: Home - Edward Donner

Text: Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and d

Content: Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Connect Four
Outsmart
An arena that pits 


## First step: Have GPT-4o-mini figure out which 'links' are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [61]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"},
    ]
}
"""

In [62]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"},
    ]
}



In [63]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [64]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascie

In [65]:
# Using OpenAI API to get related links
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [66]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 'blog/inference-providers-cohere',
 '/microsoft/bitnet-b1.58-2B-4T',
 '/HiDream-ai/HiDream-I1-Full',
 '/microsoft/MAI-DS-R1',
 '/Shakker-Labs/FLUX.1-dev-ControlNet-Union-Pro-2.0',
 '/nari-labs/Dia-1.6B',
 '/spaces/enzostvs/deepsite',
 '/spaces/bytedance-research/UNO-FLUX',
 '/spaces/InstantX/InstantCharacter',
 '/spaces/jamesliu1217/EasyControl_Ghibli',
 '/spaces/Yuanshi/OminiControl_Art',
 '/datasets/zwhe99/DeepMath-103K',
 '/datasets/openai/mrcr',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/Anthropic/values-in-the-wild',
 '/datasets/openai/graphwalks',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',
 '/docs/diffusers',
 '/docs/safetensors',
 '/docs/huggingface_hub',
 '/docs/tokenizers',
 '/docs/trl',
 '/docs/transformers.js',
 '/docs/smolagent

In [67]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another brochure prompt to GPT4-o

In [81]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [88]:
details = get_all_details('https://huggingface.co')
print(f"\nGet All Details: {details[:500]}, \n\nLength: {len(details)}")

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co/'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]} 


Get All Details: Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Posts
Docs
Enterprise
Pricing
Log In
Sign Up
NEW
Welcome Cohere on the Hub 🔥
Welcome Hyperbolic, Nebius AI Studio, and Novita on the Hub 🔥
Welcome Fireworks.ai on the Hub 🎆
The AI community building the future.
The platform where the machine learning community collabora

In [111]:
system_prompt1 = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt2 = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

system_prompt = system_prompt1

In [89]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    # user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt

In [91]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")[:5000]

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company social media', 'url': 'https://twitter.com/huggingface'}, {'type': 'company social media', 'url': 'https://www.linkedin.com/company/huggingface/'}]} 



'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nNEW\nWelcome Cohere on the Hub 🔥\nWelcome Hyperbolic, Nebius AI Studio, and Novita on the Hub 🔥\nWelcome Fireworks.ai on the Hub 🎆\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nmicrosoft/bitnet-b1.58-2B-4T\nUpdated\n2 days ago\n•\n13k\n•\n635\nHiDream-ai/HiDream-I1-Full\nUpdated\nabout 5 hours ago\n•\n24.3k\n•\n672\nmicrosoft/MAI-DS-R1\nUpdated\n5 days ago\n•\n284\n•\n194\nShakker-Labs/FLUX.1-dev-ControlNet-Union-Pro-2.0\nUpdated\n2 days

In [92]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [97]:
create_brochure("Anthropic", "https://www.anthropic.com/")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/about'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'company page', 'url': 'https://www.anthropic.com/company'}]} 



# Anthropic Company Brochure

## Company Overview
Anthropic is a leading research and development company focused on artificial intelligence (AI) and ensuring its safe and beneficial deployment. Our mission is to build AI systems that are aligned with human values, promoting understanding and reliable collaboration between humans and machines.

## Company Culture
At Anthropic, we believe in a collaborative and inclusive work environment where innovation thrives. Our team is made up of diverse talents who are passionate about AI safety and ethics. We encourage open communication, strive for continuous learning, and prioritize well-being, allowing our employees to work creatively and effectively.

- **Collaboration:** We value teamwork and are committed to working together to achieve our goals.
- **Diversity & Inclusion:** We celebrate the diverse backgrounds, perspectives, and experiences of our team members.
- **Continuous Learning:** We support personal and professional growth through opportunities for training and development.

## Customers
Anthropic serves a variety of clients across sectors that utilize AI. Our clientele includes tech companies, research institutions, and organizations focusing on responsible AI applications. We aim to create tools and research outputs that help our clients navigate the complexities of AI deployment, always with an emphasis on ethical considerations.

## Careers at Anthropic
Are you passionate about AI and its ethical implications? Join us at Anthropic to work on cutting-edge technologies that shape the future. We are always looking for skilled individuals across various domains including:

- **Research Scientists**
- **Software Engineers**
- **Policy Experts**
- **Data Analysts**

Explore the opportunity to make an impact in AI safety and join a team that cares about the future.

For more information and to view our job openings, please stay tuned to our careers page.

---

**Connect with Us!**
Follow us on social media and stay updated on our latest research, initiatives, and job opportunities.

---

*Note: This brochure is based on the limited information available, as the website pages did not contain detailed content. For comprehensive details, please visit Anthropic's official website once it is fully accessible.*

In [99]:
create_brochure("China Systems", "https://www.chinasystems.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.chinasystems.com/about'}, {'type': 'careers page', 'url': 'https://www.chinasystems.com/careers'}, {'type': 'company page', 'url': 'https://www.chinasystems.com/company'}, {'type': 'services page', 'url': 'https://www.chinasystems.com/services'}, {'type': 'contact page', 'url': 'https://www.chinasystems.com/contact'}]} 



# China Systems Brochure

## Welcome to China Systems

### Your Partner in Trade Finance Solutions

At **China Systems**, we pride ourselves on being the leading provider of innovative trade finance solutions tailored to meet the needs of our diverse clientele. Our commitment to excellence and customer satisfaction sets us apart in the industry.

---

## About Us

China Systems is dedicated to **transforming trade finance** through technology and expertise. We specialize in providing comprehensive solutions that streamline processes, mitigate risks, and enhance efficiency. Our team is comprised of seasoned professionals who understand the complexities of trade finance and are passionate about driving your business forward.

---

## Our Customers

We serve a wide range of clients, from small enterprises to large multinational corporations. Our solutions are designed to support various industries, ensuring that our customers can navigate the global trading landscape with confidence. We focus on empowering our clients by providing them with the tools and insights they need to succeed in today’s competitive market.

---

## Company Culture

At China Systems, our culture is rooted in **innovation, collaboration, and integrity**. We foster an environment where every employee is encouraged to contribute ideas, share knowledge, and grow professionally. Our commitment to teamwork ensures that we can deliver the highest quality service to our clients. Employees at China Systems enjoy a supportive workplace that values diversity and inclusion, making it a great place to build a career.

---

## Careers at China Systems

Join our dynamic team of professionals! At China Systems, we are always on the lookout for talented individuals who are passionate about trade finance and technology. Whether you are just starting your career or looking for new opportunities, we offer a range of positions that cater to various skill sets and experiences. We invest in our employees through training and development, ensuring that everyone has the tools they need to thrive.

---

## Get in Touch

We would love to hear from you! Whether you have questions about our services, want to learn more about career opportunities, or need assistance with trade finance solutions, our team is here to help. 

Visit our [contact page](#) or reach out directly to discuss how we can support your business needs.

---

### China Systems - Empowering Global Trade Through Innovative Solutions

For more information and to explore our services, please visit our website at [China Systems](#).

--- 

*This brochure is designed to provide a quick overview of our company. For detailed insights, please refer to our website or contact us directly.*

In [93]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn', 'url': 'https://www.linkedin.com/company/huggingface/'}]} 



# Hugging Face Brochure

![Hugging Face Logo](https://huggingface.co/assets/logo.svg)

## Company Overview
**Hugging Face** is at the forefront of the artificial intelligence community, building the future by democratizing machine learning (ML) and natural language processing (NLP). Founded in 2016, our mission is to create a collaborative space for researchers, developers, and organizations to engage with advanced ML technologies, fostering an open-source ecosystem that supports innovation and knowledge sharing.

## Our Offerings
### Collaborate and Innovate
- **Models**: Access a diversified selection of over **1 million models** developed by both the community and leading tech companies, including Microsoft, Google, and Amazon.
- **Datasets**: Utilize our extensive library of **250,000+ datasets** suitable for varied machine learning tasks.
- **Spaces**: Experiment with **400,000 applications** running real-time demos across text, image, and audio challenges.

### Enterprise Solutions
We provide robust enterprise solutions with:
- **Security Features**: Tailored access controls, single sign-on, and audit logs.
- **Compute Services**: Enable deployment on optimized inference endpoints, starting at just $0.60/hour.

More than **50,000 organizations** trust Hugging Face to accelerate their AI project deployments.

## Community and Culture
### A Culture of Collaboration
At Hugging Face, we embody a community-driven approach. Our platform fosters:
- **Open-Source Contributions**: Thousands of contributors improve the Hugging Face ecosystem daily.
- **Knowledge Sharing**: Engage with our community through forums, blogs, and online discussions on cutting-edge topics in AI, including reinforcement learning and optimizing AI models.

### Support and Inclusivity
We cultivate an inclusive workplace culture, where diverse perspectives are valued. Our team of **214 employees** are passionate about building accessible AI tools that enable everyone to participate in the AI revolution.

## Career Opportunities
Join us in our mission to democratize AI! We are on the lookout for passionate individuals across various roles, including:
- Software Engineering
- Machine Learning Engineering
- Product Management
- Research Roles

### Why Work at Hugging Face?
- **Innovative Environment**: Collaborate with leading experts and trailblazers in AI and ML.
- **Professional Growth**: Dive into a learning culture where you can grow and thrive.
- **Purpose-Driven**: Work on meaningful projects that shape the future of technology.

For current job openings, please visit our [Careers Page](https://huggingface.co/jobs).

## Connect with Us
Explore the Hugging Face community and collaborate today! 
- [Official Website](https://huggingface.co)
- [LinkedIn](https://www.linkedin.com/company/hugging-face)
- [Twitter](https://twitter.com/huggingface)

Together, we're building the future of AI! Join us in making machine learning accessible and beneficial for all.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

✅ With stream=True:
- Response is streamed incrementally as soon as the model starts generating output.
- You receive small chunks of the response (called deltas) in real-time, and you can start processing or displaying them immediately.
- Best for interactive or progressive rendering use cases (like chat UIs or Markdown previews).
- More responsive for long outputs.

In [105]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        # Enhancement using Stream
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    # print(f"\nDisplay Id: {display_handle.display_id}") # An unique Id
    
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        
        # 更新顯示內容
        update_display(Markdown(response), display_id=display_handle.display_id)

In [107]:
stream_brochure("China Systems", "https://www.chinasystems.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.chinasystems.com/about'}, {'type': 'careers page', 'url': 'https://www.chinasystems.com/careers'}, {'type': 'company page', 'url': 'https://www.chinasystems.com/company'}, {'type': 'services page', 'url': 'https://www.chinasystems.com/services'}]} 



# China Systems Brochure

## Welcome to China Systems
**Best Trade Finance Solutions Provider**

---

### About Us
At **China Systems**, we specialize in delivering top-notch trade finance solutions tailored to meet the unique needs of our clients. With a commitment to innovation and excellence, we pride ourselves on providing robust services that enhance the efficiency and effectiveness of international trade.

### Our Customers
We serve a diverse range of clients in the global trade sector, including multinational corporations, banks, and financial institutions. Our focus on customer satisfaction and success has made us a trusted partner in navigating the complexities of trade finance.

### Why Choose Us?
- **Expertise:** Our team comprises industry-leading professionals with deep expertise in trade finance and cutting-edge technology.
- **Innovation:** We continuously invest in research and development to offer innovative solutions that keep our clients ahead of the curve.
- **Customization:** We recognize that every business is unique, which is why we provide tailor-made solutions aligned with our clients’ specific requirements.

---

### Company Culture
At China Systems, our company culture is rooted in collaboration, integrity, and a commitment to excellence. We believe in fostering a supportive and inclusive environment where every employee can thrive and contribute to our collective success. 

- **Teamwork:** We encourage collaboration across departments to harness diverse perspectives and expertise.
- **Growth:** We prioritize professional development and provide our employees with opportunities to advance their careers within the organization.
- **Diversity:** We celebrate multiculturalism and embrace the variety of backgrounds and experiences that our team brings.

---

### Careers at China Systems
Join our dynamic workforce and be part of a company that values talent and initiative. We offer a range of career opportunities in areas such as:
- Trade Finance Solutions
- Product Development
- Marketing
- Customer Support

**Why Work With Us?**
- Competitive salaries and comprehensive benefits
- Professional development programs to enhance your skills
- A culture that supports work-life balance and personal growth

---

### Get in Touch
Discover how China Systems can transform your trade finance operations.  
**Visit our website or contact us to learn more about our services and career opportunities!**

**Website:** [www.chinasystems.com](http://www.chinasystems.com)  
**Email:** info@chinasystems.com  
**Phone:** +86 123 456 7890  

---

**Together, let's enable seamless global trade!**


Display Id: a2e9d644afc18df1e5aa6df4bc16445a


In [106]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}]} 



# Hugging Face Brochure

## About Us
**Hugging Face** is an innovative company at the forefront of artificial intelligence, dedicated to building the future of machine learning (ML) and natural language processing (NLP). Founded in 2016, we are a dynamic startup based in Paris, with a mission to democratize AI through community collaboration. Our approach combines cutting-edge technology with open-source principles, allowing users to create, discover, and share models, datasets, and applications.

---

## Our Offerings
- **Models**: Access over 1 million models across various modalities including text, images, audio, and video.
- **Datasets**: Explore a vast repository of more than 250,000 datasets tailored for machine learning tasks.
- **Spaces**: Host and collaborate on applications in our community-driven feature to foster innovation and experimentation.
- **Enterprise Solutions**: Tailored services for organizations that include enterprise-grade security, resource management, and dedicated support.

---

## Culture & Community
At Hugging Face, we believe in the power of collaboration. Our culture is built around **openness**, **innovation**, and a commitment to improving AI technologies through community-driven initiatives. We celebrate diversity and encourage contributions from a wide range of backgrounds in AI and beyond.

### Community Engagement
- **Contributions**: Join our active community of over 50,000 organizations and individual contributors shaping the future of machine learning.
- **Open Source Projects**: Engage with our open-source tools like Transformers, Diffusers, and many others, allowing developers to contribute to ML tooling.

---

## Our Customers
Our platform is trusted by over 50,000 organizations including tech giants like Microsoft, Google, Amazon, and numerous startups and research institutions. These partnerships fuel our mission to support the broader ML community while delivering valuable tools on our platform.

---

## Careers at Hugging Face
We are constantly on the lookout for passionate individuals who want to change the world through AI. Whether you're a software engineer, researcher, or product manager, we offer a vibrant work environment that fosters growth and creativity.

### Current Openings
Join our team and be a part of a groundbreaking journey in AI. Check our [careers page](https://huggingface.co/jobs) to see current openings and apply!

---

## Join Us
Become part of a vibrant community that is passionate about pushing the boundaries of technology and creating meaningful impact. Whether you're an enthusiast, a seasoned professional, or a curious observer, there's a place for you at Hugging Face.

**Explore More:**  
- [Visit our website](https://huggingface.co)  
- [Follow us on Twitter](https://twitter.com/huggingface)  
- [Join discussions in our Forums](https://huggingface.co/forums)  

--- 

Together, we are building the future of AI — one collaboration at a time!


Display Id: 6a009801e4079de8910a1bd4cca643c3


In [112]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:
system_prompt = system_prompt2

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/'}, {'type': 'company page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]} 



# 🤗 Hugging Face: The Friendship You Never Knew You Needed! 

Welcome to Hugging Face, where Artificial Intelligence meets friendly conversation! We're here to sprinkle some joy and fun into your tech journey as we build the AI future together. 🌟 

---

### **Our Mission? Hug It Out! 🤗**
At Hugging Face, we believe in democratizing AI—our goal is to make machine learning tools accessible to everyone! Whether you're data-savvy or just getting started, we’ve got something for you. 

### **Models, Data, and Fun-times! 📊**
Join the *1 Million+ Models* library, where you can browse everything from **state-of-the-art Transformers** to image-generating wizards like Gradio! Choose your adventure with **Datasets**, and explore **Spaces** to create applications that will delight you and your friends (or just impress your cat)! 

---

### **Our Customers? A Whimsical Blend! 🍰**
We serve over **50,000 organizations**, from sharp tech giants like **Amazon and Google** to non-profits working towards a better world. Think of us as the world’s cozy corner café for AI wizards to gather, share ideas, and build brilliance together. 

🔮 **Trending Models**:
- **microsoft/bitnet-b1.58-2B-4T**: Updating faster than your friend’s Instagram!
- **HiDream-ai/HiDream-I1-Full**: The ambitious model you can always count on!

---

### **Careers at Hugging Face: The Job of Your Dreams! 🌈**
Looking to join our merry band of tech enthusiasts? We’re on the hunt for adventurous minds who want to ride the waves of AI revolution! From software engineers to machine learning aficionados, if you've got a knack for innovation and a passion for collaboration, we can’t wait to chat! 🗣️ Don't worry, it’s a *hug-friendly interview process!* 🤗 

**Current Openings**: We have millions of roles no one knows about... just kidding, check out our careers page for real opportunities!

---

### **Life at Hugging Face: Culture & Community! 🎉**
Our team is a delightful mix of people who love tech, AI, and sharing a laugh. Think brains, passion, and, dare we say, some serious hugging energy (we're just a tad sentimental here)! Join our **forums** to connect with other curious minds, swap tips, or share your cool projects! 

### **Join Us! 🚀**
Whether you want to partner with us to develop groundbreaking AI models or join our vibrant team, Hugging Face is excited to welcome you to our warm, joyful community. 

**Let’s build the future together! Get started today by signing up on [Hugging Face](https://huggingface.co)!** 

---

*Remember: In a world of artificial anything, we still believe in the magic of a good hug!*


Display Id: 36966f9d0e4183ff7e1a3304b2937504


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>